In [74]:
#Importing parameters
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport parameters_julia

out_dir = parameters_julia.out_dir
files_ID = parameters_julia.files_ID
month_w = parameters_julia.month_w
year = parameters_julia.year
instances_ = parameters_julia.instances_ID


instance_ = instances_[3];

In [75]:
using JSON

instance1 = instance_

open(out_dir * "instance_comm.txt", "w") do f
    write(f, instance1)
end



#load OD pair-route incidence
odPairRoute = readstring(out_dir * "od_pair_route_incidence_" * instance1 *  files_ID * ".json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readstring(out_dir * "link_route_incidence_" * instance1 *  files_ID * ".json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readstring(out_dir *  "od_pair_label_dict_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readstring(out_dir  *"od_pair_label_dict__refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readstring(out_dir * "link_label_dict.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readstring(out_dir * "link_label_dict_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readstring(out_dir * "node_link_incidence.json");
nodeLink = JSON.parse(nodeLink);

In [76]:
include("Julia_files/load_network_uni_class.jl");

In [77]:
ta_data = load_ta_network_(out_dir, files_ID, month_w, instance1);

In [78]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [79]:
############
#Read in the demand file
file = open(out_dir * "data_traffic_assignment_uni-class/" * files_ID * "_trips_" * month_w * "_" * instance1 * ".txt")
demands = Dict()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = Int(parse(Float64,(split(line)[2])))
    elseif contains(line, ";")
        pairs = split(line, ";")
        for pair in pairs[1:end-1]
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = Int(parse(Float64,pair_vals[1])), parse(Float64,pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file);

In [80]:
demands

Dict{Any,Any} with 64 entries:
  (3, 6) => 1824.91
  (4, 4) => 0.0
  (3, 1) => 1026.72
  (4, 5) => 1152.71
  (2, 4) => 1352.56
  (6, 5) => 1438.48
  (8, 4) => 1846.1
  (1, 2) => 30.5014
  (5, 1) => 422.279
  (3, 4) => 425.322
  (8, 2) => 187.473
  (2, 3) => 6.88617
  (2, 6) => 7.80625
  (5, 4) => 598.79
  (2, 1) => 1723.62
  (2, 5) => 7.52821
  (8, 5) => 234.213
  (1, 4) => 30.372
  (2, 8) => 8.1608
  (5, 8) => 530.366
  (4, 8) => 4845.33
  (7, 8) => 1894.53
  (1, 1) => 0.0
  (7, 7) => 0.0
  (5, 7) => 503.606
  ⋮      => ⋮

In [81]:
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[odPairLabel_["$i"][1], odPairLabel_["$i"][2]]
end

In [82]:
demandsVec

56-element Array{Float64,1}:
   30.5014 
  984.779  
   30.372  
  667.846  
  670.645  
  411.881  
  436.342  
 1723.62   
    6.88617
 1352.56   
    7.52821
    7.80625
    7.72761
    ⋮      
  325.357  
  322.824  
  816.791  
  589.363  
 1894.53   
  175.974  
  187.473  
  185.953  
 1846.1    
  234.213  
  227.518  
  377.775  

In [83]:
for key=keys(odPairRoute)
    if contains(key, "56-")
        println(key)
    end
end

In [84]:
linkRoute


Dict{String,Any} with 192 entries:
  "21-61" => 1.0
  "22-81" => 1.0
  "9-8"   => 1.0
  "22-69" => 1.0
  "7-62"  => 1.0
  "7-64"  => 1.0
  "8-5"   => 1.0
  "13-63" => 1.0
  "3-61"  => 1.0
  "13-73" => 1.0
  "1-11"  => 1.0
  "15-48" => 1.0
  "7-52"  => 1.0
  "24-78" => 1.0
  "17-53" => 1.0
  "4-5"   => 1.0
  "24-74" => 1.0
  "21-75" => 1.0
  "9-33"  => 1.0
  "24-81" => 1.0
  "1-5"   => 1.0
  "2-6"   => 1.0
  "20-50" => 1.0
  "22-64" => 1.0
  "3-62"  => 1.0
  ⋮       => ⋮

In [85]:
coeffs_dict_Apr_PM_ = readstring(out_dir * "coeffs_dict_" * month_w *  "_" * instance1 * ".json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["(7, 1.5, 100.0, 1)"]
polyDeg = length(fcoeffs)

8

In [86]:
using JuMP, Ipopt

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model(solver=IpoptSolver())

numLinks = length(linkLabel_)
numRoute = length(linkRoute)
#numRoute = 400

numOD = length(demandsVec)

@variable(m, linkFlow[1:numLinks])
@variable(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @constraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()



for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @constraint(m, pathFlowLinkSum[a] == linkFlow[a])
end



for j=1:numRoute
    @NLconstraint(m, pathFlow[j] >= 0)
end


#@NLexpression(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

#@NLexpression(m, f, sum(sum(free_flow_time[a] * fcoeffs[i] * linkFlow[a]^i / capacity[a]^(i-1) for i = 1:polyDeg) 
#        for a = 1:numLinks))

@NLexpression(m, f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] +
        free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
        free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
        free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
        free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
        free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 , a = 1:numLinks})

@NLobjective(m, Min, f)
#print(m) 

solve(m);

56-82


In [87]:
getvalue(linkFlow);

In [88]:
getobjectivevalue(m)

16724.446935861524

In [89]:
flows = Dict()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getvalue(linkFlow)[i]
end

flows

Dict{Any,Any} with 24 entries:
  (3, 6) => 2503.36
  (2, 3) => 1896.84
  (5, 6) => 3823.52
  (5, 4) => 1708.89
  (2, 1) => 2620.48
  (6, 3) => 1521.97
  (3, 1) => 2411.31
  (1, 3) => 1312.76
  (4, 5) => 3775.97
  (4, 8) => 4845.33
  (7, 5) => 2857.77
  (8, 7) => 1388.91
  (3, 2) => 3831.03
  (7, 8) => 4668.87
  (2, 4) => 1382.93
  (7, 6) => 816.88
  (6, 5) => 1800.43
  (6, 7) => 3360.23
  (8, 4) => 1846.1
  (1, 2) => 1919.61
  (5, 3) => 3632.31
  (4, 2) => 2247.91
  (3, 5) => 5368.14
  (5, 7) => 3926.97

In [90]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
#using Conda
#using Pandas

@pyimport matplotlib.pyplot as plt
@pyimport numpy as np
#@pyimport json
@pyimport os
@pyimport pickle
@pyimport pandas as pd
@pyimport collections


#@pyimport pandas
#@pyimport parameters_julia
#@pyimport utils_julia
#@pyimport GLS_julia

In [91]:
@pyimport parameters_julia
@pyimport utils_julia
@pyimport GLS_julia

In [92]:
flow_user  = GLS_julia.x_;

In [93]:
function socialObj(linkFlowVec)
    objVal =  sum(sum(free_flow_time[a] * fcoeffs[i] * linkFlowVec[a]^i / capacity[a]^(i-1) for i=1:polyDeg) 
        for a = 1:numLinks)
    return objVal
end

socialObj (generic function with 1 method)

In [94]:
PoA_dict = Dict();
tapSocialFlowDicDict = Dict();
tapSocialFlowVecDict = Dict();

week_day_Apr_list = parameters_julia.week_day_list

poaDictAprPM = Dict{}()
cnt = 0
for i = 1:length(week_day_Apr_list)
    cnt +=1
    poaDictAprPM[i] = socialObj(flow_user[:, cnt])/getobjectivevalue(m)
    #println(socialObj(flow_user[:, cnt]))
    println(poaDictAprPM[i])
end


In [95]:
#=
outfile = open(out_dir * "poaDict" * month_w * instance1 * "$(day).json", "w")

JSON.print(outfile, poaDictAprPM)

close(outfile)
=#

In [96]:
flow_user

24×21 Array{Float64,2}:
 2457.32  2963.21  2865.32  2712.24  …  2283.9   2112.98  3176.2   1481.07
 1754.55  2027.93  1857.32  2480.48     1733.52  2301.19  1775.89  2360.47
 1536.14  1248.32  1522.17  1576.1      1226.37  1553.57  1373.53  1674.66
 3644.99  5777.29  2825.64  6943.2      3974.39  3350.73  2518.98  2308.13
 1438.08  2306.29  2457.99  3306.66     1986.73  1936.78  2322.32  2095.33
 2546.51  2162.25  2438.16  1905.44  …  1714.21  2283.48  2244.97  1993.12
 2031.57  1934.49  1855.14  4099.63     1541.83  3473.59  1558.5   1706.42
 3271.06  6013.17  6513.84  6972.52     6883.16  6813.33  6237.52  5874.97
 4820.06  7091.25  5241.35  6776.56     6230.99  6931.26  6027.08  6549.82
 1299.84  1561.95  1300.45  1249.53     1183.06  1087.46  1298.76  1140.65
 1820.8   1742.34  1806.53  1792.1   …  1672.59  2142.67  1447.7   2533.75
 1494.86  1333.68  1645.49  1523.98     1289.5   3770.6   1235.53  1346.45
 3575.93  3645.11  4399.74  5987.78     5565.21  6531.33  3948.94  4149.07
 

In [97]:
flows

Dict{Any,Any} with 24 entries:
  (3, 6) => 2503.36
  (2, 3) => 1896.84
  (5, 6) => 3823.52
  (5, 4) => 1708.89
  (2, 1) => 2620.48
  (6, 3) => 1521.97
  (3, 1) => 2411.31
  (1, 3) => 1312.76
  (4, 5) => 3775.97
  (4, 8) => 4845.33
  (7, 5) => 2857.77
  (8, 7) => 1388.91
  (3, 2) => 3831.03
  (7, 8) => 4668.87
  (2, 4) => 1382.93
  (7, 6) => 816.88
  (6, 5) => 1800.43
  (6, 7) => 3360.23
  (8, 4) => 1846.1
  (1, 2) => 1919.61
  (5, 3) => 3632.31
  (4, 2) => 2247.91
  (3, 5) => 5368.14
  (5, 7) => 3926.97

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      294
Number of nonzeros in inequality constraint Jacobian.:      192
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:      216
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       80
Total number of inequality constraints...............:      192
        inequality constraints with only lower bounds:      192
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 0